In [1]:
using Catlab, Catlab.Theories, Catlab.Graphics
import Catlab.Graphics: Graphviz

┌ Info: Precompiling Catlab [134e5e36-593f-5add-ad60-77f754baafbe]
└ @ Base loading.jl:1260


In [2]:
@theory SymmetricMonoidalCategory(Ob,Hom) => CategoryWithDataServices(Ob,Hom,Data) begin
    @op (u) := underlying
    
    Data::TYPE
    
    # Data service maps
    underlying(V::Data)::Ob
    μ(V::Data)::Hom(u(V) ⊗ u(V), u(V))
    δ(V::Data)::Hom(u(V), u(V) ⊗ u(V))
    ϵ(V::Data)::Hom(u(V), munit())
    
    # Data service axioms
    (μ(V) ⊗ id(u(V))) ⋅ μ(V) == (id(u(V)) ⊗ μ(V)) ⋅ μ(V) ⊣ (V::Data) # associativity of μ
    braid(u(V), u(V)) ⋅ μ(V) == μ(V) ⊣ (V::Data) # commutativity of μ
    δ(V) ⋅ (δ(V) ⊗ id(u(V))) == δ(V) ⋅ (id(u(V)) ⊗ δ(V)) ⊣ (V::Data) # associativity of δ
    δ(V) ⋅ (ϵ(V) ⊗ id(u(V))) == id(u(V)) ⊣ (V::Data) # unitality of δ
    δ(V) ⋅ (id(u(V)) ⊗ ϵ(V)) == id(u(V)) ⊣ (V::Data) # "
    δ(V) ⋅ braid(u(V), u(V)) == δ(V) ⊣ (V::Data) # commutativity of δ
    (δ(V) ⊗ id(u(V))) ⋅ (id(u(V)) ⊗ μ(V)) == μ(V) ⋅ δ(V) ⊣ (V::Data) # Frobenius law
    (id(u(V)) ⊗ δ(V)) ⋅ (μ(V) ⊗ id(u(V))) == μ(V) ⋅ δ(V) ⊣ (V::Data) # "
    δ(V) ⋅ μ(V) == id(u(V)) ⊣ (V::Data) # Special axiom
end

@theory CategoryWithDataServices(Ob,Hom,Data) => CategoryWithAttributes(Ob,Hom,Data,AttrOb,AttrHom) begin
    @op (e) := entity
    @op (v) := value
    
    AttrOb::TYPE
    AttrHom(dom::AttrOb,codom::AttrOb)::TYPE
    
    # All of the functions we need to have a category with attributes
    compose(f::AttrHom(A,B), g::AttrHom(B,C))::AttrHom(A,C) ⊣ (A::AttrOb, B::AttrOb, C::AttrOb)
    id(A::AttrOb)::AttrHom(A,A)
    entity(A::AttrOb)::Ob
    entity(f::AttrHom(A,B))::Hom(e(A), e(B)) ⊣ (A::AttrOb, B::AttrOb)
    value(A::AttrOb)::Data
    value(f::AttrHom(A,B))::Hom(u(v(A)), u(v(B))) ⊣ (A::AttrOb, B::AttrOb)
    γ(A::AttrOb)::Hom(e(A), e(A) ⊗ u(v(A)))
    
    # Category axioms for the category of attributes
    (f ⋅ g) ⋅ h == f ⋅ (g ⋅ h) ⊣ (A::AttrOb, B::AttrOb, C::AttrOb, D::AttrOb, f::AttrHom(A, B), g::AttrHom(B, C), h::AttrHom(C, D))
    id(A) ⋅ f == f ⊣ (A::AttrOb, B::AttrOb, f::AttrHom)
    f ⋅ id(B) == f ⊣ (A::AttrOb, B::AttrOb, f::AttrHom)
    
    # Functoriality of e
    e(id(A)) == id(e(A)) ⊣ (A::AttrOb)
    e(f ⋅ g) == e(f) ⋅ e(g) ⊣ (A::AttrOb, B::AttrOb, C::AttrOb, f::AttrHom(A,B), g::AttrHom(B,C))
    
    # Functoriality of v
    v(id(A)) == id(u(v(A))) ⊣ (A::AttrOb)
    v(f ⋅ g) == v(f) ⋅ v(g) ⊣ (A::AttrOb, B::AttrOb, C::AttrOb, f::AttrHom(A,B), g::AttrHom(B,C))
    
    # Axioms for γ
    γ(A) ⋅ (e(f) ⊗ v(f)) == e(f) ⋅ γ(B) ⊣ (A::AttrOb, B::AttrOb, f::AttrHom(A,B)) # Naturality of γ
    γ(A) ⋅ (id(e(A)) ⊗ δ(v(A))) == γ(A) ⋅ (γ(A) ⊗ id(u(v(A)))) ⊣ (A::AttrOb)
    γ(A) ⋅ (id(e(A)) ⊗ ϵ(v(A))) == id(e(A)) ⊣ (A::AttrOb)
end

CategoryWithAttributes

In [3]:
# "Syntax" / free categories with gadgets
# Patterned off of implementations of same in Catlab

abstract type DataExpr{T} <: CategoryExpr{T} end
abstract type AttrObExpr{T} <: CategoryExpr{T} end
abstract type AttrHomExpr{T} <: CategoryExpr{T} end

@syntax FreeCategoryWithDataServices(ObExpr, HomExpr, DataExpr) CategoryWithDataServices begin
    compose(f::Hom, g::Hom) = begin
        if f == id(dom(g))
            return g
        elseif g == id(codom(f))
            return f
        else
            return associate(new(f,g; strict=true)) # def. used in Catlab
        end
    end
    otimes(A::Ob, B::Ob) = associate_unit(new(A, B), munit)
    otimes(f::Hom, g::Hom) = associate(new(f, g))
    # do we need to throw in rules for the others?
end

@syntax FreeCategoryWithAttributes(ObExpr, HomExpr, DataExpr, AttrObExpr, AttrHomExpr) CategoryWithAttributes begin
    compose(f::Hom, g::Hom) = begin
        if f == id(dom(g))
            return g
        elseif g == id(codom(f))
            return f
        else
            return associate(new(f,g; strict=true)) # def. used in Catlab
        end
    end
    compose(f::AttrHom, g::AttrHom) = begin
        if f == id(dom(g))
            return g
        elseif g == id(codom(f))
            return f
        else
            return associate(new(f,g; strict=true)) # def. used in Catlab
        end
    end
    otimes(A::Ob, B::Ob) = associate_unit(new(A, B), munit)
    otimes(f::Hom, g::Hom) = associate(new(f, g))
end

Main.FreeCategoryWithAttributes

In [4]:
@present RobotExample(FreeCategoryWithAttributes) begin
    Robot::Ob
    Ball::Ob
    RobotWithBall::Ob
    
    Location::Data
    LocationSquared::Data
    
    MoveTo::Hom(Robot ⊗ u(Location), Robot)
    MoveTo2::Hom(RobotWithBall ⊗ u(Location), RobotWithBall)
    Pick::Hom(Robot ⊗ Ball, RobotWithBall)
    Place::Hom(RobotWithBall, Robot ⊗ Ball)
    
    RxLLocation::AttrOb
    RLocation::AttrOb
    RwBxLLocation::AttrOb
    RwBLocation::AttrOb
    RxBLocation::AttrOb

    AttrMoveTo::AttrHom(RxLLocation, RLocation)
    AttrMoveTo2::AttrHom(RwBxLLocation, RwBLocation)
    AttrPick::AttrHom(RxBLocation, RwBLocation)
    AttrPlace::AttrHom(RwBLocation, RxBLocation)
    
    u(LocationSquared) == u(Location) ⊗ u(Location)
    
    (μ(Location) ⊗ id(u(Location))) ⋅ μ(Location) == (id(u(Location)) ⊗ μ(Location)) ⋅ μ(Location)
    braid(u(Location), u(Location)) ⋅ μ(Location) == μ(Location)
    δ(Location) ⋅ (δ(Location) ⊗ id(u(Location))) == δ(Location) ⋅ (id(u(Location)) ⊗ δ(Location))
    δ(Location) ⋅ (ϵ(Location) ⊗ id(u(Location))) == id(u(Location))
    δ(Location) ⋅ (id(u(Location)) ⊗ ϵ(Location)) == id(u(Location))
    δ(Location) ⋅ braid(u(Location), u(Location)) == δ(Location)
    (δ(Location) ⊗ id(u(Location))) ⋅ (id(u(Location)) ⊗ μ(Location)) == μ(Location) ⋅ δ(Location)
    (id(u(Location)) ⊗ δ(Location)) ⋅ (μ(Location) ⊗ id(u(Location))) == μ(Location) ⋅ δ(Location)
    δ(Location) ⋅ μ(Location) == id(u(Location))
    
    e(RxLLocation) == Robot ⊗ u(Location)
    e(RLocation) == Robot
    e(RwBxLLocation) == RobotWithBall ⊗ u(Location)
    e(RwBLocation) == RobotWithBall
    e(RxBLocation) == Robot ⊗ Ball
    
    e(AttrMoveTo) == MoveTo
    e(AttrMoveTo2) == MoveTo2
    e(AttrPick) == Pick
    e(AttrPlace) == Place
    
    v(RxLLocation) == LocationSquared
    v(RLocation) == Location
    v(RwBxLLocation) == LocationSquared
    v(RwBLocation) == Location
    v(RxBLocation) == LocationSquared
    
    v(AttrMoveTo) == ϵ(Location) ⊗ id(u(Location))
    v(AttrMoveTo2) == ϵ(Location) ⊗ id(u(Location))
    v(AttrPick) == μ(Location)
    v(AttrPlace) == δ(Location)
    
    γ(RxLLocation) ⋅ (id(e(RxLLocation)) ⊗ δ(v(RxLLocation))) == γ(RxLLocation) ⋅ (γ(RxLLocation) ⊗ id(u(v(RxLLocation))))
    γ(RxLLocation) ⋅ (id(e(RxLLocation)) ⊗ ϵ(v(RxLLocation))) == id(e(RxLLocation))
    γ(RLocation) ⋅ (id(e(RLocation)) ⊗ δ(v(RLocation))) == γ(RLocation) ⋅ (γ(RLocation) ⊗ id(u(v(RLocation))))
    γ(RLocation) ⋅ (id(e(RLocation)) ⊗ ϵ(v(RLocation))) == id(e(RLocation))
    γ(RwBxLLocation) ⋅ (id(e(RwBxLLocation)) ⊗ δ(v(RwBxLLocation))) == γ(RwBxLLocation) ⋅ (γ(RwBxLLocation) ⊗ id(u(v(RwBxLLocation))))
    γ(RwBxLLocation) ⋅ (id(e(RwBxLLocation)) ⊗ ϵ(v(RwBxLLocation))) == id(e(RwBxLLocation))
    γ(RwBLocation) ⋅ (id(e(RwBLocation)) ⊗ δ(v(RwBLocation))) == γ(RwBLocation) ⋅ (γ(RwBLocation) ⊗ id(u(v(RwBLocation))))
    γ(RwBLocation) ⋅ (id(e(RwBLocation)) ⊗ ϵ(v(RwBLocation))) == id(e(RwBLocation))
    γ(RxBLocation) ⋅ (id(e(RxBLocation)) ⊗ δ(v(RxBLocation))) == γ(RxBLocation) ⋅ (γ(RxBLocation) ⊗ id(u(v(RxBLocation))))
    γ(RxBLocation) ⋅ (id(e(RxBLocation)) ⊗ ϵ(v(RxBLocation))) == id(e(RxBLocation))
    
    γ(RxLLocation) ⋅ (e(AttrMoveTo) ⊗ v(AttrMoveTo)) == e(AttrMoveTo) ⋅ γ(RLocation)
    γ(RwBxLLocation) ⋅ (e(AttrMoveTo2) ⊗ v(AttrMoveTo2)) == e(AttrMoveTo2) ⋅ γ(RwBLocation)
    γ(RxBLocation) ⋅ (e(AttrPick) ⊗ v(AttrPick)) == e(AttrPick) ⋅ γ(RwBLocation)
    γ(RwBLocation) ⋅ (e(AttrPlace) ⊗ v(AttrPlace)) == e(AttrPlace) ⋅ γ(RxBLocation)
end

Presentation{CategoryWithAttributes,Symbol}(Main.FreeCategoryWithAttributes, (Ob = Main.FreeCategoryWithAttributes.Ob{:generator}[Robot, Ball, RobotWithBall], Hom = Main.FreeCategoryWithAttributes.Hom{:generator}[MoveTo, MoveTo2, Pick, Place], Data = Main.FreeCategoryWithAttributes.Data{:generator}[Location, LocationSquared], AttrOb = Main.FreeCategoryWithAttributes.AttrOb{:generator}[RxLLocation, RLocation, RwBxLLocation, RwBLocation, RxBLocation], AttrHom = Main.FreeCategoryWithAttributes.AttrHom{:generator}[AttrMoveTo, AttrMoveTo2, AttrPick, AttrPlace]), Dict(:Location => (:Data => 1),:AttrPlace => (:AttrHom => 4),:RobotWithBall => (:Ob => 3),:RxLLocation => (:AttrOb => 1),:RLocation => (:AttrOb => 2),:AttrMoveTo2 => (:AttrHom => 2),:AttrPick => (:AttrHom => 3),:Pick => (:Hom => 3),:Ball => (:Ob => 2),:RwBxLLocation => (:AttrOb => 3)…), Pair[underlying(LocationSquared) => otimes(underlying(Location),underlying(Location)), compose(otimes(μ(Location),id(underlying(Location))),μ(Locati

In [ ]:
# TODO:
# Add a few more equations for the γ's and data service structures above
# Figure out how to create wiring diagrams for categories with attributes
# See if there's a way to shorten the above presentation
# See if there's actually anything interesting that can be done with the above presentation